In [2]:
import pandas as pd

# Define la ruta del archivo Excel
ruta_archivo = 'C:\\Users\\lbarc\\OneDrive - Universidad Pontificia Comillas\\ICAI 11\\Machine Learning III\\Datos Dia 1\\STRIKE_REPORTS.xlsx'

# Carga el archivo Excel en un DataFrame de pandas
df = pd.read_excel(ruta_archivo, sheet_name='STRIKE_REPORTS')


  INCIDENT_DATE TIME_OF_DAY AIRPORT_ID STATE FAAREGION         OPERATOR  \
0    1996-06-22         NaN       KSMF    CA       AWP  UNITED AIRLINES   
1    1996-06-26         NaN       KDEN    CO       ANM  UNITED AIRLINES   
2    1996-07-01         NaN       KOMA    NE       ACE  UNITED AIRLINES   
3    1996-07-01         NaN       KIAD    DC       AEA  UNITED AIRLINES   
4    1996-07-01         NaN       KLGA    NY       AEA  UNITED AIRLINES   

    AIRCRAFT AC_CLASS  AC_MASS TYPE_ENG  ...  PRECIPITATION AOS  COST_REPAIRS  \
0  B-737-300        A      4.0        D  ...            NaN NaN           NaN   
1  B-737-300        A      4.0        D  ...            NaN NaN           NaN   
2  B-757-200        A      4.0        D  ...            NaN NaN           NaN   
3      A-320        A      4.0        D  ...            NaN NaN           NaN   
4      A-320        A      4.0        D  ...            NaN NaN           NaN   

   COST_OTHER  COST_REPAIRS_INFL_ADJ COST_OTHER_INFL_ADJ EFFEC

In [3]:
df = df.dropna(subset=['HEIGHT', 'SPEED', 'DISTANCE', 'NUM_STRUCK'])
df.head()

,INCIDENT_DATE,TIME_OF_DAY,AIRPORT_ID,STATE,FAAREGION,OPERATOR,AIRCRAFT,AC_CLASS,AC_MASS,TYPE_ENG,...,PRECIPITATION,AOS,COST_REPAIRS,COST_OTHER,COST_REPAIRS_INFL_ADJ,COST_OTHER_INFL_ADJ,EFFECT,SPECIES,WARNED,NUM_STRUCK
5,1991-05-06,Day,KSAT,TX,ASW,AMERICAN AIRLINES,B-727-100,A,4.0,D,...,None,NaN,NaN,NaN,NaN,NaN,None,Unknown bird - medium,No,1
11,1990-09-07,Day,KAGS,GA,ASO,UNKNOWN COMMERCIAL,DC-9,A,4.0,D,...,Fog,NaN,NaN,NaN,NaN,NaN,NaN,Unknown bird - small,No,2-10
13,1993-12-08,Day,KDFW,TX,ASW,AMERICAN AIRLINES,MD-11,A,4.0,D,...,Fog,NaN,NaN,NaN,NaN,NaN,None,Unknown bird - small,No,1
16,1990-07-13,Day,KLAX,CA,AWP,ALASKA AIRLINES,MD-80,A,4.0,D,...,None,NaN,NaN,NaN,NaN,NaN,NaN,American kestrel,No,1
18,1995-04-28,NaN,KSFO,CA,AWP,UNITED AIRLINES,B-757-200,A,4.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,Aborted Take-off,Unknown bird - medium,Unknown,11-100


In [4]:
# Nombre del archivo de salida
nombre_archivo_salida = 'datos_limpios.xlsx'

# Exporta el DataFrame limpio a un archivo Excel
df.to_excel(nombre_archivo_salida, index=False)

print(f'Archivo guardado como: {nombre_archivo_salida}')

Archivo guardado como: datos_limpios.xlsx
